In [1]:
# library imports:

import pandas as pd
from pathlib import Path
import portiloop_software
import torch
from portiloop_software import run_offline_unlabelled, get_final_model_config_dict, get_trained_model
from matplotlib import pyplot as plt
from torchsummary import summary
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv1D, MaxPool1D, GRU
from tensorflow.keras import Model
import numpy as np

In [2]:
# path to the portiloop software package:

path_software = Path(portiloop_software.__file__).parent.absolute()

In [3]:
# path to the folder containing pre-trained models:

path_experiments = path_software / 'experiments'

In [4]:
# configuration dictionary of the model:

config_dict = get_final_model_config_dict()

In [5]:
# run offline inference (on all data points):

model_torch = get_trained_model(config_dict, path_experiments)

In [6]:
model_torch

PortiloopNetwork(
  (first_layer_input1): ConvPoolModule(
    (conv): Conv1d(1, 31, kernel_size=(7,), stride=(1,))
    (pool): MaxPool1d(kernel_size=7, stride=1, padding=0, dilation=1, ceil_mode=False)
    (dropout): Dropout(p=0, inplace=False)
  )
  (seq_input1): Sequential(
    (0): ConvPoolModule(
      (conv): Conv1d(31, 31, kernel_size=(7,), stride=(1,))
      (pool): MaxPool1d(kernel_size=7, stride=1, padding=0, dilation=1, ceil_mode=False)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): ConvPoolModule(
      (conv): Conv1d(31, 31, kernel_size=(7,), stride=(1,))
      (pool): MaxPool1d(kernel_size=7, stride=1, padding=0, dilation=1, ceil_mode=False)
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
  (gru_input1): GRU(558, 7, batch_first=True)
  (fc): Linear(in_features=7, out_features=1, bias=True)
)

In [7]:
summary(model_torch)

Layer (type:depth-idx)                   Param #
├─ConvPoolModule: 1-1                    --
|    └─Conv1d: 2-1                       248
|    └─MaxPool1d: 2-2                    --
|    └─Dropout: 2-3                      --
├─Sequential: 1-2                        --
|    └─ConvPoolModule: 2-4               --
|    |    └─Conv1d: 3-1                  6,758
|    |    └─MaxPool1d: 3-2               --
|    |    └─Dropout: 3-3                 --
|    └─ConvPoolModule: 2-5               --
|    |    └─Conv1d: 3-4                  6,758
|    |    └─MaxPool1d: 3-5               --
|    |    └─Dropout: 3-6                 --
├─GRU: 1-3                               11,907
├─Linear: 1-4                            8
Total params: 25,679
Trainable params: 25,679
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─ConvPoolModule: 1-1                    --
|    └─Conv1d: 2-1                       248
|    └─MaxPool1d: 2-2                    --
|    └─Dropout: 2-3                      --
├─Sequential: 1-2                        --
|    └─ConvPoolModule: 2-4               --
|    |    └─Conv1d: 3-1                  6,758
|    |    └─MaxPool1d: 3-2               --
|    |    └─Dropout: 3-3                 --
|    └─ConvPoolModule: 2-5               --
|    |    └─Conv1d: 3-4                  6,758
|    |    └─MaxPool1d: 3-5               --
|    |    └─Dropout: 3-6                 --
├─GRU: 1-3                               11,907
├─Linear: 1-4                            8
Total params: 25,679
Trainable params: 25,679
Non-trainable params: 0

In [8]:
model_keras = tf.keras.Sequential()
model_keras.add(tf.keras.layers.Reshape((-1, 54, 1)))
model_keras.add(tf.keras.layers.Conv1D(31, strides=[1], kernel_size=7, activation='relu'))
model_keras.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 7), strides=1, padding='valid'))
model_keras.add(tf.keras.layers.Conv1D(31, strides=[1], kernel_size=7, activation='relu'))
model_keras.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 7), strides=1, padding='valid'))
model_keras.add(tf.keras.layers.Conv1D(31, strides=[1], kernel_size=7, activation='relu'))
model_keras.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 7), strides=1, padding='valid'))
model_keras.add(tf.keras.layers.Reshape((-1, 558)))
model_keras.add(GRU(units=7, time_major=False))
model_keras.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model_keras.build((None, 50, 54))
model_keras.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 50, 54, 1)         0         
                                                                 
 conv1d (Conv1D)             (None, 50, 48, 31)        248       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 42, 31)       0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 36, 31)        6758      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 50, 30, 31)       0         
 2D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 50, 24, 31)        6

In [9]:
model_keras.layers

In [10]:
torch_params = [param.detach().numpy() for param in model_torch.parameters()]

In [11]:
for i, param in enumerate(torch_params):
    print(f"param {i}: {param.shape}")

param 0: (31, 1, 7)
param 1: (31,)
param 2: (31, 31, 7)
param 3: (31,)
param 4: (31, 31, 7)
param 5: (31,)
param 6: (21, 558)
param 7: (21, 7)
param 8: (21,)
param 9: (21,)
param 10: (1, 7)
param 11: (1,)


In [12]:
def convert_kernel_inv(kernel):
    kernel_r, kernel_z, kernel_h = np.hsplit(kernel, 3)
#     print(f"kernel_r:{kernel_r}")
#     print(f"kernel_z:{kernel_z}")
#     print(f"kernel_h:{kernel_h}")
    return np.concatenate((kernel_z.T, kernel_r.T, kernel_h.T))

def convert_kernel(kernel):
    kernel_z, kernel_r, kernel_h = np.vsplit(kernel, 3)
#     print(f"kernel_r:{kernel_r}")
#     print(f"kernel_z:{kernel_z}")
#     print(f"kernel_h:{kernel_h}")
    return np.concatenate((kernel_r.T, kernel_z.T, kernel_h.T), axis=1)

def convert_bias(bias):
    bias = bias.reshape(2, 3, -1) 
    return bias[:, [1, 0, 2], :].reshape((2, -1))

In [14]:
model_keras.layers[8].weights

[<tf.Variable 'gru/gru_cell/kernel:0' shape=(558, 21) dtype=float32, numpy=
 array([[-0.00516923, -0.08542526, -0.06721321, ...,  0.06840996,
          0.01528582,  0.02371211],
        [ 0.01809502,  0.02903289,  0.04205378, ..., -0.0908815 ,
          0.01830979, -0.05411907],
        [-0.02839405, -0.07618035, -0.01555226, ..., -0.02818249,
         -0.05247126,  0.04197291],
        ...,
        [-0.02865159, -0.04985338,  0.08722272, ...,  0.00496975,
          0.0955235 , -0.08958738],
        [ 0.06841332,  0.04971463, -0.05475862, ..., -0.00147787,
          0.06389888, -0.00395075],
        [-0.05564988, -0.08756593,  0.08360628, ..., -0.02685148,
          0.01573873,  0.0937896 ]], dtype=float32)>,
 <tf.Variable 'gru/gru_cell/recurrent_kernel:0' shape=(7, 21) dtype=float32, numpy=
 array([[ 0.19350421, -0.11237555, -0.03487422,  0.531398  ,  0.14133207,
          0.20267361, -0.33967176,  0.077941  ,  0.11865258, -0.11841632,
         -0.2623738 ,  0.38621965,  0.10601867,  

In [15]:
np.stack((torch_params[8], torch_params[9]), axis=0)

array([[-0.02185846, -0.14705312, -0.06134333, -0.21139653, -0.18442982,
         0.00979619, -0.14749926, -0.24829413,  1.0149685 ,  0.30653724,
        -0.07180233, -0.6962207 , -0.48068357, -0.37287533, -0.01478305,
         0.0281016 , -0.29703212, -0.05942979,  0.1469805 ,  0.30490357,
         0.17594711],
       [-0.05141691, -0.14167127, -0.00835179, -0.31697038,  0.17389293,
         0.12304442,  0.03572496, -0.31770235,  0.99044317,  0.0597805 ,
        -0.47962168, -0.7721744 , -0.4003129 , -0.18330953,  0.1975466 ,
         0.06060247,  0.20888945,  0.0802715 , -0.32268295,  0.0396922 ,
        -0.1638425 ]], dtype=float32)

In [19]:
torch_params[6].shape

(21, 558)

In [20]:
torch_params[7].shape

(21, 7)

In [16]:
model_keras.layers[1].set_weights([torch_params[0].T, torch_params[1].T])

# Second Conv Layer:
model_keras.layers[3].set_weights([torch_params[2].T, torch_params[3].T])

# Third Conv Layer:
model_keras.layers[5].set_weights([torch_params[4].T, torch_params[5].T])

# GRU Layer:

kernel_input = convert_kernel(torch_params[6])
kernel_h = convert_kernel(torch_params[7])
bias = convert_bias(np.stack((torch_params[8], torch_params[9]), axis=0))

model_keras.layers[8].set_weights([kernel_input, 
                                   kernel_h, 
                                   bias])

# Dense Layer:
model_keras.layers[9].set_weights([torch_params[10].T, torch_params[11].T])

In [17]:
model_keras.layers[8].weights

[<tf.Variable 'gru/gru_cell/kernel:0' shape=(558, 21) dtype=float32, numpy=
 array([[-0.16189827, -0.02721401, -0.00131223, ..., -0.0224664 ,
         -0.10324092,  0.9781653 ],
        [-0.04380907,  0.07253738, -0.03930822, ..., -0.04212598,
         -0.05580707,  0.70585686],
        [-0.22128211,  0.04012021, -0.01671478, ..., -0.07443257,
         -0.06322375,  0.50437504],
        ...,
        [-0.11483102, -0.11880854, -0.21867967, ..., -0.04734865,
         -0.17091021,  0.05709471],
        [-0.38440633, -0.02999607, -0.02425925, ..., -0.0608215 ,
         -0.324488  ,  0.04215567],
        [-0.3089492 , -0.13285387,  0.00966667, ..., -0.04424573,
         -0.311438  ,  0.10411689]], dtype=float32)>,
 <tf.Variable 'gru/gru_cell/recurrent_kernel:0' shape=(7, 21) dtype=float32, numpy=
 array([[-0.25731143,  0.20413998,  0.03256913, -0.12562902, -0.09498331,
         -0.04637064,  0.57656753, -0.6201707 ,  0.11997185, -0.385144  ,
         -0.16755357, -0.01406851, -0.30629292,  

In [21]:
input_numpy = np.ones((1, 50, 54))
input_torch = torch.ones((1, 50, 54))

In [22]:
x, hn1, _, _ = model_torch(input_torch, None, None, torch.zeros(1, 1, 7), None)

DEBUG: =========================
DEBUG: x.shape:(50, 31, 42)
DEBUG: x:
[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.0898011  0.0898011  0.0898011  ... 0.0898011  0.0898011  0.0898011 ]
  ...
  [0.02003773 0.02003773 0.02003773 ... 0.02003773 0.02003773 0.02003773]
  [0.20981292 0.20981292 0.20981292 ... 0.20981292 0.20981292 0.20981292]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.0898011  0.0898011  0.0898011  ... 0.0898011  0.0898011  0.0898011 ]
  ...
  [0.02003773 0.02003773 0.02003773 ... 0.02003773 0.02003773 0.02003773]
  [0.20981292 0.20981292 0.20981292 ... 0.20981292 0.20981292 0.20981292]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         

In [23]:
x

tensor([[0.0500]], grad_fn=<SigmoidBackward0>)

In [24]:
hn1

tensor([[[ 0.7966, -0.0322,  0.4980, -0.0575, -0.2911, -0.0938,  0.9845]]],
       grad_fn=<StackBackward0>)

In [25]:
out_keras = model_keras(input_numpy)
print(out_keras)

tf.Tensor([[0.09079182]], shape=(1, 1), dtype=float32)


In [26]:
from keras import backend as K

inp = model_keras.input                                           # input placeholder
outputs = [layer.output for layer in model_keras.layers]          # all layer outputs
functors = [K.function([inp], [out]) for out in outputs]   # evaluation function

# Testing
layer_outs = [func([input_numpy]) for func in functors]

In [27]:
print("===")
print(np.array(layer_outs[2]).squeeze().swapaxes(1,2).shape)
print(np.array(layer_outs[2]).squeeze().swapaxes(1,2))

print("===")
print(np.array(layer_outs[4]).squeeze().swapaxes(1,2).shape)
print(np.array(layer_outs[4]).squeeze().swapaxes(1,2))

print("===")
print(np.array(layer_outs[6]).squeeze().swapaxes(1,2).shape)
print(np.array(layer_outs[6]).squeeze().swapaxes(1,2))

print("===")
print(np.array(layer_outs[8]).squeeze().shape)
print(np.array(layer_outs[8]).squeeze())

===
(50, 31, 42)
[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.0898011  0.0898011  0.0898011  ... 0.0898011  0.0898011  0.0898011 ]
  ...
  [0.02003773 0.02003773 0.02003773 ... 0.02003773 0.02003773 0.02003773]
  [0.20981297 0.20981297 0.20981297 ... 0.20981297 0.20981297 0.20981297]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.0898011  0.0898011  0.0898011  ... 0.0898011  0.0898011  0.0898011 ]
  ...
  [0.02003773 0.02003773 0.02003773 ... 0.02003773 0.02003773 0.02003773]
  [0.20981297 0.20981297 0.20981297 ... 0.20981297 0.20981297 0.20981297]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.